#### Libraries

In [18]:
import helper_functions_2 as hf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import glob
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%reload_ext autoreload
%autoreload 2

#### Image calibration

In [2]:
objpoints, imgpoints = hf.calibration()

#### Load the test images

In [3]:
test_images = hf.image_loader()

#### Undistort the images

In [4]:
undistorted_ones = []
for each_image in test_images:
    undistorted_one = hf.undistort(each_image, objpoints, imgpoints)
    undistorted_ones.append(undistorted_one)

# I have all the images now in undistorted ones array

#### Image Processing : Color selection, gradients,  perspective transform and Polynomial Fit

In [19]:
def pipeline(image):
    # 1. The channel Selector
    h,l,s = hf.hls_selector(image)
    
    #2. the Color Thresholding
    color = hf.color_threshold(s, color_t=(170,255))
    
    #3. The gradients and its thresholds
    sobely = hf.sobel_y(s, thresh_min=40, thresh_max=255)
    sobelx = hf.sobel_x(s, thresh_min=10, thresh_max=100)
    
    #4. Combination Of binary outputs
    combined = np.zeros_like(sobelx)
    combined = np.dstack(( sobely, color, sobelx))*255
    
    #5. Perspective Transform
    warped_image, m_inv = hf.perspective_transform(combined)
    
    #6. Logic for Polynomial processing
    
    left_fitx, right_fitx, ploty, left_pos, right_pos = hf.fit_polynomial(warped_image)
    
    
    out_img = hf.draw_lines(warped_image, image, m_inv, left_fitx, right_fitx, ploty)
    
    left_line.radius_of_curvature_meters = int(hf.measure_curvature_real()[0])
    right_line.radius_of_curvature_meters = int(hf.measure_curvature_real()[1])
    avg_curv = (left_line.radius_of_curvature_meters + right_line.radius_of_curvature_meters) // 2
    
    curv_s = 'Radius of Curvature = {}(m)'.format(avg_curv)
    cv2.putText(out_img, curv_s, (30,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255))
    
    center_car = image.shape[1] // 2
    
    l_c = int(right_pos - left_pos)
    offset = (l_c - center_car) * (3.7/700)
    
    
    if (offset > 0):
        offset_string = 'Vehicle is {0:.2f}m left of center'.format(offset)
    else:
        offset_string = 'Vehicle is {0:.2f}m right of center'.format(np.absolute(offset))
        
    cv2.putText(out_img, offset_string, (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255))
    
    return out_img

In [20]:
vid_output = 'output_images/video_test4.mp4'
clip = VideoFileClip("project_video.mp4")
white_clip = clip.fl_image(pipeline) 
%time white_clip.write_videofile(vid_output, audio=False)

[MoviePy] >>>> Building video output_images/video_test4.mp4
[MoviePy] Writing video output_images/video_test4.mp4


100%|█████████▉| 1260/1261 [05:02<00:00,  4.23it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/video_test4.mp4 

CPU times: user 2min 29s, sys: 26.4 s, total: 2min 56s
Wall time: 5min 5s


In [21]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(vid_output))